In [ ]:
# %pip install openai
# %pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 26.8 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 40.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 28.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

dataset_dir = 'datasets'

# read the dataset
df = pd.read_csv(f'{dataset_dir}/podcast.csv')
df.head()

,text,text_length,sentiment,word_count,unique_word_count,pos_tags,entities,clean_text
0,"As part of MIT course 6S099, Artificial Genera...",74068,0.171118,13424,2726,"[('As', 'IN'), ('part', 'NN'), ('of', 'IN'), (...","[('MIT', 'ORG'), ('Artificial General Intellig...",part mit course artificial general intelligenc...
1,As part of MIT course 6S099 on artificial gene...,57788,0.116574,10217,2474,"[('As', 'IN'), ('part', 'NN'), ('of', 'IN'), (...","[('MIT', 'ORG'), ('Christoph Koch', 'PERSON'),...",part mit course artificial general intelligenc...
2,"You've studied the human mind, cognition, lang...",33940,0.106534,5989,1762,"[('You', 'PRP'), (""'ve"", 'VBP'), ('studied', '...","[('Plato', 'ORG'), ('B.', 'NORP'), ('Nietzsche...",studied human mind cognition language vision e...
3,What difference between biological neural netw...,34228,0.106092,5993,1528,"[('What', 'WP'), ('difference', 'NN'), ('betwe...","[('First', 'ORDINAL'), ('One', 'CARDINAL'), ('...",difference biological neural networks artifici...
4,The following is a conversation with Vladimir ...,36171,0.125654,6374,1481,"[('The', 'DT'), ('following', 'NN'), ('is', 'V...","[('Vladimir Vapnik', 'PERSON'), ('VC', 'PERSON...",following conversation vladimir vapnik co inve...


In [4]:
# we only want the text column
df = df[['text']]
df = df.dropna()

In [ ]:
from openai import OpenAI

api_key = ""

client = OpenAI(
    api_key=api_key,
)

In [ ]:
prompt = "Write a short summary, limited to 40 words, for the following long form text: "

def generate_summary(text):
    max_text_length = 2000
    if len(text) > max_text_length:
        text = text[:max_text_length]
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt + "\"" + text + "\"",
                }
            ],
            model="gpt-3.5-turbo",
        )
        response_summary = chat_completion.choices[0].message.content
        # remove "Summary:" from the response
        response_summary = response_summary.replace("Summary:", "")
        return response_summary
    except Exception as e:
        print(f"Error generating summary for text: {e}")
        return None

# generate row 1 summary
print(generate_summary(df['text'][1]))

df['summary'] = df['text'].apply(generate_summary)

Sit down with neurobiologist Christoph Koch, president of the Allen Institute for Brain Science. Known for his impact on consciousness studies, he shares insights on intelligent life in the universe and the uniqueness of human existence.


In [15]:
df.head()

,text,summary
0,"As part of MIT course 6S099, Artificial Genera...",An enlightening conversation with physicist M...
1,As part of MIT course 6S099 on artificial gene...,MIT course 6S099 on artificial general intelli...
2,"You've studied the human mind, cognition, lang...",Exploring the meaning of life through the lens...
3,What difference between biological neural netw...,The mystery of how biological neural networks ...
4,The following is a conversation with Vladimir ...,"Vladimir Vapnik, a renowned statistician and ..."


In [16]:
# save the dataset
df.to_csv(f'{dataset_dir}/podcast_with_summary.csv', index=False)